<a href="https://colab.research.google.com/github/teamgaon/recommendation_algorithms_know/blob/main/20220111_hj_know_rf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

import pandas as pd
from glob import glob
from tqdm import tqdm
import warnings

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
warnings.filterwarnings(action='ignore') 
know_train = [pd.read_csv(path) for path in sorted(glob('/content/drive/MyDrive/KNOW/train/train/*.csv'))]
know_test = [pd.read_csv(path) for path in sorted(glob('/content/drive/MyDrive/KNOW/test/test/*.csv'))]

In [4]:
for df in know_train:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [5]:
for df in know_test:
    for col in df.columns:
        df[col].replace(' ', '0', inplace=True)

In [6]:
a = know_train[0]
a['bq4_1a']

0        자동차도장기능사
1       건축전기설비기술사
2       건축전기설비기술사
3            지적기사
4       건축전기설비기술사
          ...    
9481            0
9482            0
9483            0
9484            0
9485            0
Name: bq4_1a, Length: 9486, dtype: object

In [7]:
years = ['2017', '2018', '2019', '2020']

rf_predicts = []
year_encoder = {}

for year, train, test in zip(years, know_train, know_test):
    print(year)
    encoders = {}
    for col in train.columns:
        if col == 'ID':
            continue
        try:
            train[col] = train[col].map(int)
        except:
            encoder = LabelEncoder()
            train[col] = train[col].map(str)
            train[col] = encoder.fit_transform(train[col])
            encoders[col] = encoder  
    year_encoder[year] = encoders
    feature = train.columns.drop(['idx', 'knowcode'])
    model = RandomForestClassifier(n_estimators=100, random_state=123456, n_jobs=8)
    model.fit(train[feature], train['knowcode'])

    for col in test.columns:
      try:
          test[col] = test[col].map(int)
      except:
          encoder = year_encoder[year][col]
          test[col] = test[col].map(str)
          category_map = {category: idx for idx, category in enumerate(encoder.classes_)}
          test[col] = test[col].apply(lambda x: category_map[x] if x in category_map else -1) # train set에서 보지못한 카테고리변수 -1(UNK) 처리
    feature = test.columns.drop(['idx'])
    pred = model.predict(test[feature])
    rf_predicts.extend(pred)

2017
2018
2019
2020


In [10]:
submission = pd.read_csv('/content/drive/MyDrive/KNOW/sample_submission.csv') # sample submission 불러오기

In [11]:
submission['knowcode'] = rf_predicts

In [12]:
submission

,idx,knowcode
0,0,29401
1,1,155303
2,2,22103
3,3,121102
4,4,412003
...,...,...
35226,35244,851101
35227,35245,29303
35228,35246,833001
35229,35247,140101


In [13]:
submission.to_csv('submission.csv', index=False)

In [14]:
train_know

NameError: ignored